In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_Papillary_Cell_Carcinoma/GSE85258'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Differential gene expression profiling of matched primary renal cell carcinoma and metastases reveals upregulation of extracellular matrix genes"
!Series_summary	"It is unknown if gene expression profiles from primary RCC tumors differ from patient-matched metastatic tumors. Thus, we sought to identify differentially expressed genes between patient-matched primary and metastatic RCC tumors in order to understand the molecular mechanisms underlying the development of RCC metastases."
!Series_overall_design	"We compared gene expression profiles between patient-matched primary and metastatic RCC tumors. We employed Affymetrix U133 Plus2 microarrays to assess gene expression on 15 pairs of primary RCC (14 clear cell RCC, 1 papillary) tumors and patient-matched pulmonary metastases, across 54,675 probesets."
Sample Characteristics Dictionary:
{0: ['patient: 1', 'patient: 2', 'patient: 3', 'patient: 4', 'patient: 5', 'patient: 6', 'patient: 7', 'patient

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Sample Characteristics Dictionary (Defined previously)
sample_characteristics = {
    0: ['patient: 1', 'patient: 2', 'patient: 3', 'patient: 4', 'patient: 5', 'patient: 6', 'patient: 7', 'patient: 8', 'patient: 9', 'patient: 10', 'patient: 11', 'patient: 12', 'patient: 13', 'patient: 14', 'patient: 15'],
    1: ['tissue: Mets', 'tissue: Primary'],
    2: ['primary subtype: ccRCC', 'primary subtype: Papillary RCC']
}

# Check for gene expression data availability
is_gene_available = True  # Assuming Affymetrix U133 Plus2 microarrays data is gene expression data

# Assign keys for available variables
trait_row = 2 if 'Papillary RCC' in [val.split(': ')[1] for val in sample_characteristics[2]] else None
age_row = None  # No data regarding age found in sample characteristics dictionary
gender_row = None  # No data regarding gender found in sample characteristics dictionary

# Define conversion functions
def convert_trait(value):
    val = value.split(': ')[1]
    if val == 'ccRCC':
        return 0
    elif val == 'Papillary RCC':
        return 1
    return None

convert_age = lambda x: None  # Age data not available
convert_gender = lambda x: None  # Gender data not available

# Save cohort information
save_cohort_info('GSE85258', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if trait_row is not None
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_Papillary_Cell_Carcinoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/trait_data/GSE85258.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2263272': [0], 'GSM2263273': [0], 'GSM2263274': [0], 'GSM2263275': [0], 'GSM2263276': [0], 'GSM2263277': [0], 'GSM2263278': [1], 'GSM2263279': [1], 'GSM2263280': [0], 'GSM2263281': [0], 'GSM2263282': [0], 'GSM2263283': [0], 'GSM2263284': [0], 'GSM2263285': [0], 'GSM2263286': [0], 'GSM2263287': [0], 'GSM2263288': [0], 'GSM2263289': [0], 'GSM2263290': [0], 'GSM2263291': [0], 'GSM2263292': [0], 'GSM2263293': [0], 'GSM2263294': [0], 'GSM2263295': [0], 'GSM2263296': [0], 'GSM2263297': [0], 'GSM2263298': [0], 'GSM2263299': [0], 'GSM2263300': [0], 'GSM2263301': [0], 'GSM2263302': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
if requires_gene_mapping:
    # 1. Determine the keys for identifiers and gene symbols
    identifier_key = 'ID'
    gene_symbol_key = 'Gene Symbol'
    
    # 2. Get the dataframe containing the mapping between probe IDs and genes
    gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    
    # 3. Apply the mapping to get the gene expression dataframe
    gene_data = apply_gene_mapping(gene_data, gene_mapping_df)

# At this point, "gene_data" should be the gene expression dataframe with gene symbols as identifiers.


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE85258.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Kidney_Papillary_Cell_Carcinoma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE85258', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/GSE85258.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Kidney_Papillary_Cell_Carcinoma', the least common label is '1.0' with 2 occurrences. This represents 6.45% of the dataset.
The distribution of the feature 'Kidney_Papillary_Cell_Carcinoma' in this dataset is severely biased.

